##### Cell 1 项目“启动配置单元”，它保证路径干净、环境正确、数据已找到，为后续分析做好准备。

In [2]:
from pathlib import Path
import os

# 检查当前工作目录
print("当前工作目录:", Path.cwd())

# ✅ 如果当前不在 notebooks 文件夹内，则切换过去
target_dir = Path("/Users/xuzhiwei/Desktop/SleepQualityProject/notebooks")
if Path.cwd() != target_dir:
    os.chdir(target_dir)
    print("🔄 已自动切换到:", Path.cwd())

# 定义项目根目录与子文件夹路径
ROOT = Path.cwd().parents[0]                 # → SleepQualityProject
KAGGLE_DIR = ROOT / "Kaggle_SleepHealth"     # Kaggle 数据文件夹
EDF_DIR = ROOT / "Sleep_EDF"                 # Sleep EDF 文件夹
REPORTS = ROOT / "reports"                   # 输出统计结果
FIGS = ROOT / "figures"                      # 输出图像

# 创建输出文件夹（如不存在）
for p in [REPORTS, FIGS]:
    p.mkdir(parents=True, exist_ok=True)

# ✅ 检查 Kaggle 文件夹是否存在
if not KAGGLE_DIR.exists():
    raise FileNotFoundError(f"❌ 没找到文件夹: {KAGGLE_DIR}")

# ✅ 检查是否存在 CSV 文件
csv_files = sorted(KAGGLE_DIR.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"❌ Kaggle_SleepHealth 文件夹内没有找到任何 .csv 文件！\n路径: {KAGGLE_DIR}")
CSV_PATH = csv_files[0]

# 输出路径确认信息
print("✅ 数据路径设置完成！")
print("项目根目录:", ROOT)
print("Kaggle 文件路径:", CSV_PATH)
print("报告输出路径:", REPORTS)
print("图像输出路径:", FIGS)

当前工作目录: /Users/xuzhiwei/Desktop/SleepQualityProject/notebooks
✅ 数据路径设置完成！
项目根目录: /Users/xuzhiwei/Desktop/SleepQualityProject
Kaggle 文件路径: /Users/xuzhiwei/Desktop/SleepQualityProject/Kaggle_SleepHealth/Sleep_health_and_lifestyle_dataset.csv
报告输出路径: /Users/xuzhiwei/Desktop/SleepQualityProject/reports
图像输出路径: /Users/xuzhiwei/Desktop/SleepQualityProject/figures


#### Cell 2 负责准备分析环境 —— 确保所有常用的科学计算库已正确加载、版本正常，并让 Notebook 的输出排版更友好。

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 显示当前库版本（确认环境兼容）
print("✅ 已成功导入依赖")
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("matplotlib version:", plt.matplotlib.__version__)

# 设置一些显示选项（让输出更清晰）
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 100)

✅ 已成功导入依赖
pandas version: 2.2.3
numpy version: 2.1.3
matplotlib version: 3.10.0


In [4]:
# 使用 Cell1 中定义好的 CSV_PATH
print("正在读取数据文件：", CSV_PATH)

# 1️⃣ 读取 CSV
df = pd.read_csv(CSV_PATH)

# 2️⃣ 标准化列名（去空格、统一小写、用下划线连接）
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(r"[^\w]+", "_", regex=True)
)

# 3️⃣ 查看数据形状（行 × 列）
rows, cols = df.shape
print(f"✅ 已成功读取数据：{rows} 行 × {cols} 列")

# 4️⃣ 打印前几行样本（确认数据内容）
display(df.head(5))

# 5️⃣ 可选：显示列名与数据类型，了解每列的类型（数值 / 分类 / 文本）
print("\n📋 数据列名与类型：")
print(df.dtypes)

# 6️⃣ 保存一份标准化后的快照（可选，方便之后重复使用）
df.to_csv(REPORTS / "kaggle_cleaned_snapshot.csv", index=False)
print("\n📁 已保存标准化后的快照至:", REPORTS / "kaggle_cleaned_snapshot.csv")

正在读取数据文件： /Users/xuzhiwei/Desktop/SleepQualityProject/Kaggle_SleepHealth/Sleep_health_and_lifestyle_dataset.csv
✅ 已成功读取数据：374 行 × 13 列


,person_id,gender,age,occupation,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,bmi_category,blood_pressure,heart_rate,daily_steps,sleep_disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea



📋 数据列名与类型：
person_id                    int64
gender                      object
age                          int64
occupation                  object
sleep_duration             float64
quality_of_sleep             int64
physical_activity_level      int64
stress_level                 int64
bmi_category                object
blood_pressure              object
heart_rate                   int64
daily_steps                  int64
sleep_disorder              object
dtype: object

📁 已保存标准化后的快照至: /Users/xuzhiwei/Desktop/SleepQualityProject/reports/kaggle_cleaned_snapshot.csv


#### Cell 3 的核心价值是确认数据载入正确、结构清晰、类型可用。它让你在进入统计分析（缺失值、异常值检测）前，明确知道“你手里拿的是什么样的数据”。

In [5]:
# 使用 Cell1 中定义好的 CSV_PATH
print("正在读取数据文件：", CSV_PATH)

# 1️⃣ 读取 CSV
df = pd.read_csv(CSV_PATH)

# 2️⃣ 标准化列名（去空格、统一小写、用下划线连接）
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(r"[^\w]+", "_", regex=True)
)

# 3️⃣ 查看数据形状（行 × 列）
rows, cols = df.shape
print(f"✅ 已成功读取数据：{rows} 行 × {cols} 列")

# 4️⃣ 打印前几行样本（确认数据内容）
display(df.head(5))

# 5️⃣ 可选：显示列名与数据类型，了解每列的类型（数值 / 分类 / 文本）
print("\n📋 数据列名与类型：")
print(df.dtypes)

# 6️⃣ 保存一份标准化后的快照（可选，方便之后重复使用）
df.to_csv(REPORTS / "kaggle_cleaned_snapshot.csv", index=False)
print("\n📁 已保存标准化后的快照至:", REPORTS / "kaggle_cleaned_snapshot.csv")

正在读取数据文件： /Users/xuzhiwei/Desktop/SleepQualityProject/Kaggle_SleepHealth/Sleep_health_and_lifestyle_dataset.csv
✅ 已成功读取数据：374 行 × 13 列


,person_id,gender,age,occupation,sleep_duration,quality_of_sleep,physical_activity_level,stress_level,bmi_category,blood_pressure,heart_rate,daily_steps,sleep_disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea



📋 数据列名与类型：
person_id                    int64
gender                      object
age                          int64
occupation                  object
sleep_duration             float64
quality_of_sleep             int64
physical_activity_level      int64
stress_level                 int64
bmi_category                object
blood_pressure              object
heart_rate                   int64
daily_steps                  int64
sleep_disorder              object
dtype: object

📁 已保存标准化后的快照至: /Users/xuzhiwei/Desktop/SleepQualityProject/reports/kaggle_cleaned_snapshot.csv


#### cell 4 的作用就是把“缺失值问题”数字化、存证化，给后续清洗提供依据。
#### 量化缺失：算出整表缺失率（你的是 4.5%）与含任意缺失的行比例（219/374 = 58.56%）。
#### 定位问题列：按列给出缺失数与缺失率，并导出到 reports/kaggle_missingness.csv，还画了 Top-10 柱状图到 figures/missing_top10.png。
#### 可复查：所有数字都落地到文件，后面报告/表格直接引用，不用手抄。

In [6]:
import numpy as np

# 1) 每列缺失统计
missing = df.isna().sum().to_frame("n_missing")
missing["missing_%"] = (missing["n_missing"] / len(df) * 100).round(2)
missing = missing.sort_values("missing_%", ascending=False)

# 2) 全局/逐行缺失概览
overall_missing_pct = round(
    (missing["n_missing"].sum()) / (len(df) * len(df.columns)) * 100, 2
)  # 所有单元格层面
rows_with_any_na = int(df.isna().any(axis=1).sum())
rows_with_any_na_pct = round(rows_with_any_na / len(df) * 100, 2)

# 3) 导出报告
out_path = REPORTS / "kaggle_missingness.csv"
missing.to_csv(out_path)

# 4) 可视化（Top 10 缺失列）
top = missing.head(10)
ax = top["missing_%"].plot(kind="bar", rot=45, title="Top-10 Missing (%)")
ax.set_ylabel("Missing (%)")
fig = ax.get_figure()
(figures_path := (FIGS / "missing_top10.png"))
fig.savefig(figures_path, dpi=150, bbox_inches="tight")
plt.close(fig)

# 5) 输出关键信息
print("✅ 缺失值统计完成并已导出")
print(f"- Overall missingness (cell-level): {overall_missing_pct}%")
print(f"- Rows with any NA: {rows_with_any_na} / {len(df)} ({rows_with_any_na_pct}%)")
print(f"- Report saved to: {out_path}")
print(f"- Figure saved to: {figures_path}")

# 6) 展示前几项
display(missing.head(15))

✅ 缺失值统计完成并已导出
- Overall missingness (cell-level): 4.5%
- Rows with any NA: 219 / 374 (58.56%)
- Report saved to: /Users/xuzhiwei/Desktop/SleepQualityProject/reports/kaggle_missingness.csv
- Figure saved to: /Users/xuzhiwei/Desktop/SleepQualityProject/figures/missing_top10.png


,n_missing,missing_%
sleep_disorder,219,58.56
person_id,0,0.00
gender,0,0.00
age,0,0.00
occupation,0,0.00
sleep_duration,0,0.00
quality_of_sleep,0,0.00
physical_activity_level,0,0.00
stress_level,0,0.00
bmi_category,0,0.00


#### Cell 5 是数值特征的健康体检报告，告诉你数据分布是否正常、是否偏斜、是否需要特殊填补或缩放，为后续的 异常值检测 (Cell 6) 提供基准参考。

In [7]:
# 📋 Cell 5: 重复行与数值列概览（导出报告）
# 依赖变量：df, REPORTS
# -----------------------------

import numpy as np

# 1) 统计重复行
n_dups = int(df.duplicated().sum())
dup_rate = round(n_dups / len(df) * 100, 2)
print(f"✅ Duplicate rows: {n_dups} / {len(df)} ({dup_rate}%)")

# 2) 选择数值列
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
print("数值列:", num_cols)

# 3) 数值列汇总（均值/中位数/标准差/极值/缺失率/偏度）
summary = df[num_cols].agg(["mean", "median", "std", "min", "max"]).T
summary["mean_median_gap"] = (summary["mean"] - summary["median"]).abs()
summary["missing_%"] = (df[num_cols].isna().sum() / len(df) * 100).round(2)
# 偏度（判断偏态，便于选择 median vs mean）
summary["skew"] = df[num_cols].skew(numeric_only=True)

# 4) 导出报告
out_path = REPORTS / "kaggle_numeric_summary.csv"
summary.to_csv(out_path)

# 5) 结果预览
print("📁 Summary saved to:", out_path)
display(summary.sort_values("mean_median_gap", ascending=False).head(10))

✅ Duplicate rows: 0 / 374 (0.0%)
数值列: ['person_id', 'age', 'sleep_duration', 'quality_of_sleep', 'physical_activity_level', 'stress_level', 'heart_rate', 'daily_steps']
📁 Summary saved to: /Users/xuzhiwei/Desktop/SleepQualityProject/reports/kaggle_numeric_summary.csv


,mean,median,std,min,max,mean_median_gap,missing_%,skew
daily_steps,6816.844920,7000.0,1617.915679,3000.0,10000.0,183.155080,0.0,0.178277
physical_activity_level,59.171123,60.0,20.830804,30.0,90.0,0.828877,0.0,0.074487
age,42.184492,43.0,8.673133,27.0,59.0,0.815508,0.0,0.257222
stress_level,5.385027,5.0,1.774526,3.0,8.0,0.385027,0.0,0.154330
quality_of_sleep,7.312834,7.0,1.196956,4.0,9.0,0.312834,0.0,-0.207448
heart_rate,70.165775,70.0,4.135676,65.0,86.0,0.165775,0.0,1.224824
sleep_duration,7.132086,7.2,0.795657,5.8,8.5,0.067914,0.0,0.037554
person_id,187.500000,187.5,108.108742,1.0,374.0,0.000000,0.0,0.000000


#### Cell 6 是异常值体检表 —— 它让你知道哪些数值偏离正常范围、偏离多少、是否需要在后续建模前清洗或调整。 数据整体健康，仅 4% 样本在心率上偏离。没有明显的系统性问题，可继续进入 Week 10 的特征处理与建模阶段

In [8]:
# 📈 Cell 6: IQR 异常值检测 + 图表导出
# 依赖：df, REPORTS, FIGS（来自前面单元）
# 功能：对所有数值列用 IQR（四分位距）法检测离群点并导出报告与图像
# -----------------------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) 选择数值列
num_cols = df.select_dtypes(include="number").columns.tolist()
assert len(num_cols) > 0, "当前数据集中没有数值列可用于 IQR 检测。"

# 2) IQR 统计函数
def iqr_stats(s: pd.Series) -> pd.Series:
    s = s.dropna()
    if s.empty:
        return pd.Series({
            "Q1": np.nan, "Q3": np.nan, "IQR": np.nan,
            "lower_fence": np.nan, "upper_fence": np.nan,
            "n_outliers": 0, "pct_outliers": 0.0
        })
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    mask = (s < lower) | (s > upper)
    return pd.Series({
        "Q1": q1, "Q3": q3, "IQR": iqr,
        "lower_fence": lower, "upper_fence": upper,
        "n_outliers": int(mask.sum()),
        "pct_outliers": round(mask.mean()*100, 2)
    })

# 3) 计算每个数值列的 IQR 离群统计表
iqr_table = pd.DataFrame({col: iqr_stats(df[col]) for col in num_cols}).T
iqr_table = iqr_table.sort_values("pct_outliers", ascending=False)

# 4) 导出报告
out_csv = REPORTS / "kaggle_iqr_outliers.csv"
iqr_table.to_csv(out_csv)

# 5) 可视化：Top-10 离群率柱状图
top = iqr_table.head(10)
ax = top["pct_outliers"].plot(kind="bar", rot=45, title="Top-10 IQR Outliers (%)")
ax.set_ylabel("% of outliers")
fig = ax.get_figure()
(fig_bar := FIGS / "iqr_outliers_top10.png")
fig.savefig(fig_bar, dpi=150, bbox_inches="tight")
plt.close(fig)

# 6) 为每个数值列保存一张盒须图（便于肉眼查看离群分布）
for col in num_cols:
    ax = df.boxplot(column=col)
    ax.set_title(f"{col} boxplot (IQR fences)")
    fig = ax.get_figure()
    fig.savefig(FIGS / f"box_{col}.png", dpi=150, bbox_inches="tight")
    plt.close(fig)

# 7) 输出关键信息
total_outliers = int(iqr_table["n_outliers"].sum())
print("✅ IQR 异常值检测完成")
print(f"- 数值列：{len(num_cols)} 个 → {num_cols}")
print(f"- 总离群点数：{total_outliers}")
print(f"- 报告已保存：{out_csv}")
print(f"- 图像已保存：{fig_bar} 以及 figures/box_*.png")

# 8) 预览前几行结果
display(iqr_table.head(10))

✅ IQR 异常值检测完成
- 数值列：8 个 → ['person_id', 'age', 'sleep_duration', 'quality_of_sleep', 'physical_activity_level', 'stress_level', 'heart_rate', 'daily_steps']
- 总离群点数：15
- 报告已保存：/Users/xuzhiwei/Desktop/SleepQualityProject/reports/kaggle_iqr_outliers.csv
- 图像已保存：/Users/xuzhiwei/Desktop/SleepQualityProject/figures/iqr_outliers_top10.png 以及 figures/box_*.png


,Q1,Q3,IQR,lower_fence,upper_fence,n_outliers,pct_outliers
heart_rate,68.00,72.00,4.00,62.000,78.000,15.0,4.01
person_id,94.25,280.75,186.50,-185.500,560.500,0.0,0.00
age,35.25,50.00,14.75,13.125,72.125,0.0,0.00
sleep_duration,6.40,7.80,1.40,4.300,9.900,0.0,0.00
quality_of_sleep,6.00,8.00,2.00,3.000,11.000,0.0,0.00
physical_activity_level,45.00,75.00,30.00,0.000,120.000,0.0,0.00
stress_level,4.00,7.00,3.00,-0.500,11.500,0.0,0.00
daily_steps,5600.00,8000.00,2400.00,2000.000,11600.000,0.0,0.00


#### Cell 7 是“数据清洗阶段的收尾单元” —— 它让你的数据在进入建模前彻底干净：
#### 异常值已处理；
#### 高缺失列已标注；
#### 输出两个可靠数据版本（删除 vs 截断）；
#### 所有结果都存档，可追溯可展示。

In [9]:
# 🧼 Cell 7: 异常值处理方案（两版）+ 缺失指示器 + 导出
# 依赖：df, iqr_table, REPORTS, FIGS
# 说明：
#   - 生成两份“可选的”清洗结果（仍属 Week 9，不做建模）：
#       A) drop：删除含 IQR 异常值的行（保守）
#       B) winsorize：对异常值按 IQR 边界进行截断/折顶（稳健）
#   - 对 sleep_disorder 仅新增缺失指示器，不做填补（高缺失率）
# -----------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 1) 选出需要进行异常处理的数值列（排除标识列）
numeric_cols = df.select_dtypes(include="number").columns.tolist()
id_like = {"person_id", "id", "index"}
cols_to_consider = [c for c in numeric_cols if c not in id_like]

# 只保留实际存在异常值的列
cols_with_outliers = []
for c in cols_to_consider:
    if c in iqr_table.index and iqr_table.loc[c, "n_outliers"] > 0:
        cols_with_outliers.append(c)

print("需处理的数值列（含 IQR 异常）:", cols_with_outliers if cols_with_outliers else "无")

# 2) 准备 IQR 边界
fences = {}
for c in cols_with_outliers:
    row = iqr_table.loc[c]
    fences[c] = (row["lower_fence"], row["upper_fence"])

# 3) 版本 A：删除含异常值的行（任何被判定为 outlier 的列都删除）
df_drop = df.copy()
for c in cols_with_outliers:
    lo, up = fences[c]
    mask = (df_drop[c] < lo) | (df_drop[c] > up)
    df_drop = df_drop.loc[~mask]

removed_rows = len(df) - len(df_drop)

# 4) 版本 B：winsorize（把异常值截断到 IQR 边界）
df_cap = df.copy()
for c in cols_with_outliers:
    lo, up = fences[c]
    df_cap[c] = df_cap[c].clip(lower=lo, upper=up)

# 5) 为两份数据添加缺失指示器（sleep_disorder_missing），不填补原列
for _df in (df_drop, df_cap):
    if "sleep_disorder" in _df.columns:
        _df["sleep_disorder_missing"] = _df["sleep_disorder"].isna().astype(int)

# 6) 导出两份候选清洗结果
out_drop = REPORTS / "kaggle_clean_drop_outliers.csv"
out_cap  = REPORTS / "kaggle_clean_winsorized.csv"
df_drop.to_csv(out_drop, index=False)
df_cap.to_csv(out_cap, index=False)

# 7) 可选：对关键列（若存在）画一个“前后对比”直方图（以 heart_rate 为例）
key_col = "heart_rate"
if key_col in cols_with_outliers:
    plt.figure()
    df[key_col].plot(kind="hist", bins=20, alpha=0.5, label="original")
    df_cap[key_col].plot(kind="hist", bins=20, alpha=0.5, label="winsorized")
    plt.title(f"{key_col}: before vs winsorized")
    plt.legend()
    fig_path = FIGS / f"hist_{key_col}_before_after.png"
    plt.savefig(fig_path, dpi=150, bbox_inches="tight")
    plt.close()
else:
    fig_path = None

# 8) 输出摘要
print("✅ 清洗候选版本已生成：")
print(f"- 删除异常版 (drop) → {out_drop}  | 移除行数: {removed_rows}")
print(f"- 截断异常版 (winsorize) → {out_cap}")
if fig_path:
    print(f"- 直方图对比图已保存：{fig_path}")

# 9) 小贴士（给后续周使用的选择建议）
print("\n📌 建议：")
print("- 若样本量充足并担心极端值是录入误差 → 使用 drop 版本。")
print("- 若希望保留样本、但降低极端值影响 → 使用 winsorize 版本。")
print("- sleep_disorder 高缺失：目前仅添加缺失指示器，不做填补；作为目标变量时建议先筛选非缺失子集。")

需处理的数值列（含 IQR 异常）: ['heart_rate']
✅ 清洗候选版本已生成：
- 删除异常版 (drop) → /Users/xuzhiwei/Desktop/SleepQualityProject/reports/kaggle_clean_drop_outliers.csv  | 移除行数: 15
- 截断异常版 (winsorize) → /Users/xuzhiwei/Desktop/SleepQualityProject/reports/kaggle_clean_winsorized.csv
- 直方图对比图已保存：/Users/xuzhiwei/Desktop/SleepQualityProject/figures/hist_heart_rate_before_after.png

📌 建议：
- 若样本量充足并担心极端值是录入误差 → 使用 drop 版本。
- 若希望保留样本、但降低极端值影响 → 使用 winsorize 版本。
- sleep_disorder 高缺失：目前仅添加缺失指示器，不做填补；作为目标变量时建议先筛选非缺失子集。
